https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

# Word-Level Text Generator in Keras

In this project, I will develop a word level text generator in Keras using LSTM. I will train it on a Harry Potter dataset.

In [1]:
import string
from numpy import array

import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.callbacks import LambdaCallback

from pickle import dump

in_filename = '../datasets/harry-potter-1.txt'
out_filename = 'harry-potter.txt'
input_size = 50
output_size = 1

Using TensorFlow backend.


## Load the Document

The first step in creating the model is to load the corpus into memory.

In [2]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

Once we've got the function to load the document, we find the path to the corpus that we are using, which in this case is:

`../datasets/harry-potter-1.txt`

In [3]:
# load document
doc = load_doc(in_filename)
print(doc[:200])

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or


## Clean the Document

Now that we've loaded the document into memory, we want to clean the document. For example, before splitting the document into words, we may want to replace all "-" with spaces to so that the words split more nicely. We also take out the punctuation from each word.

In [4]:
# turn a doc into clean tokens
def clean_doc(doc):
    # make lower case
    doc = doc.lower()
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    doc = doc.replace('-', ' ')
    doc = doc.replace('. ', ' . ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    punctuation = string.punctuation.replace(".", "")
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha() or word=="."]
    return tokens

We can then run the cleaning function on the document that we've stored in memory.

In [5]:
# clean document
tokens = clean_doc(doc)
print(tokens[:20])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['mr', '.', 'and', 'mrs', '.', 'dursley', 'of', 'number', 'four', 'privet', 'drive', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal']
Total Tokens: 81694
Unique Tokens: 5867


## Dataset Preparation

Now that we've tokenized our data (that is, separate the document into the list of words), we can now organize the dataset into input and output words. In this case, we've set the input to be 50 words followed by the next word. In otherwords, sequences of length 51, or `input_size + output_size`.

The resulting list, `sequences`, is a list of strings with only 51 words each.

In [31]:
# organize into sequences of tokens
length = input_size + output_size
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 81643


In addition, we also create a function that will export the above list of strings to a separate document.

In [32]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

Now we call the function

In [33]:
# save sequences to file
save_doc(sequences, out_filename)

# Training The Model

Now that we've done the data preparation, we can load the dataset and really prepare it for the model by one hot encoding.

## Load the Sequences

First, we load the file!

In [48]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# load
doc = load_doc("harry-potter.txt")
lines = doc.split('\n')
lines[0]

'mr . and mrs . dursley of number four privet drive were proud to say that they were perfectly normal thank you very much . they were the last people youd expect to be involved in anything strange or mysterious because they just didnt hold with such nonsense . mr .'

## Encode the Sequences

Now, before we can train the model on the data, we need to tokenize the data again. We load the Tokenizer, and use it to prepare the lines. We save these tokenized sentences into an array of sequences. 

In [72]:
# remove punctuation from each token
punctuation = string.punctuation.replace(".", "")
# integer encode sequences of words
tokenizer = Tokenizer(filters=punctuation)
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Now let's take a quick look at the vocabulary size of the model. This basically shows how many words the model has available to pick from when predicting the next word.

We'll also store this vocabulary size into a variable `vocab_size` to use in defining our model later.

In [69]:
js

'[".","the","and","to","a","he","of","harry","was","it","in","his","you","said","had","i","on","at","they","that","as","him","but","with","ron","all","out","for","be","up","what","were","hagrid","them","have","there","hermione","back","one","if","from","this","so","its","not","she","about","into","an","their","been","got","me","know","off","didnt","like","could","get","no","just","down","professor","her","when","who","see","is","very","over","looked","do","then","now","are","we","by","your","dont","around","dumbledore","snape","hed","well","how","going","never","go","through","think","something","look","right","uncle","im","time","harrys","first","dudley","only","looking","more","or","would","malfoy","my","even","couldnt","door","eyes","before","here","vernon","did","neville","too","head","two","where","again","yeh","next","thought","mcgonagall","than","way","come","told","still","people","face","other","quirrell","which","room","will","potter","though","good","because","last","left","

In [58]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

5868


Now that we've encoded the data, we need to separate the dataset into input `X` and output `y` elements.

In [59]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

## Fit the Model

Now, we get to the meat of things. We are going to define the model that we are going to use

In [60]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# define model
model = Sequential()
model.add(Embedding(vocab_size, input_size, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            293400    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 5868)              592668    
Total params: 1,036,968
Trainable params: 1,036,968
Non-trainable params: 0
_________________________________________________________________
None


Finally, the model is ready to be fit on the data for some amount of epochs. This takes a few hours even on modern hardware without gpu's. You can speed up the training by increasing the `batch_size` or decreasing the number of `epochs`.

We then also save this model. This is the point where the tutorial that I've been following ends, but personally I like to save my models periodically, so before we train, I'm going to define a callback function which will checkpoint the model every five epochs in case I get impatient. This utilizes the keras `LambdaCallback` class. 

In [63]:
def on_epoch_end (epoch, _):
    if (epoch + 1) % 5 == 0:
        print("Checkpointing the model...")
        model.save(".h5")
        # save the tokenizer
        dump(tokenizer, open('tokenizer.pkl', 'wb'))
        
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [61]:
# fit model
model.fit(X, y, batch_size=256, epochs=20, verbose=2, callbacks=[print_callback])

Epoch 1/2
81643/81643 [==============================] - 180s 2ms/step - loss: 6.5243 - acc: 0.0584
Epoch 2/2
81643/81643 [==============================] - 175s 2ms/step - loss: 6.2286 - acc: 0.0622


In [70]:
# save the model to file
model.save('model.h5')

js = "["
for word, index in tokenizer.word_index.items():
    js += '"' + word + '",'
js += "]"
f = open("tokenizer-array.txt", "w")
f.write(js)

55157

# Using the Model!

## Load the Models and the Tokenizer

In [55]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# load cleaned text sequences
in_filename = 'harry-potter.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

seq_length = len(lines[0].split()) - 1

In [56]:
from keras.models import load_model
from pickle import load

# load the model
model = load_model('model.h5')
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

## Generate Text

In [57]:
from random import randint

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

arms around the trolls neck from behind the troll couldnt feel harry hanging there but even a troll will notice if you stick a long bit of wood up its nose and harrys wand had still been in his hand when hed jumped it had gone straight up one of the



In [58]:
encoded = tokenizer.texts_to_sequences([seed_text])[0]
print(encoded)

[456, 80, 1, 1218, 574, 41, 150, 1, 371, 109, 530, 7, 800, 35, 22, 108, 4, 371, 136, 1132, 40, 12, 995, 4, 172, 181, 6, 228, 27, 44, 365, 2, 98, 195, 14, 130, 52, 10, 11, 178, 66, 83, 744, 9, 14, 235, 424, 27, 38, 6, 1]


In [62]:
# truncate sequences to a fixed length
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
# predict probabilities for each word
yhat = model.predict_classes(encoded, verbose=0)

In [63]:
# map predicted word index to word
out_word = ''
for word, index in tokenizer.word_index.items():
    if index == yhat:
        out_word = word
        break

In [65]:

# append to input
seed_text += ' ' + out_word
result.append(out_word)

NameError: name 'result' is not defined

## Complete Code For Loading and Using

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

# load cleaned text sequences
in_filename = 'harry-potter.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)